# Nh5 Event Model

## Notes on building an event model based in hdf5, pytabes and pandas

In [1]:
from __future__ import print_function

### GATE event model

GATE event model (EM) can be exposed to python (via pyROOT). On the other hand:

1. Since the GATE objects were designed in C++, the objects exposed to python are "C++ translations" performed by pyROOT. This comes at the cost of loosing reflection/introspection. pyROOT objects can only be manipulated using their C++ interfaces, which are often not obviously translated to python (maps is a good example). 

2. GATE objects are stored using ROOT. Depending on your faith you may consider this a blessing or a curse. 

3. A description of GATE EVM is here:

http://next.ific.uv.es:8888/nextsw/dstbuilder/wikis/EventModel

## Nh5 Event Model

### Raw Data objects

#### True Waveforms (TWF), Raw Waveforms (RWF),  Baseline restored Waveforms (BRWF) 

1. TWF are waveforms of photoelectrons (pes) produced by NEXUS and include detector response simulated by Geant-4, propagation of drift electrons, generation and propagation of light, geometrical effects in the sensors (windows) and the QE/PDE of the sensors (PMTs and SiPMs). TWF do not include the response of the electronics. 

2. For detailed studies of the Energy Plane FEE, very thinly binned TWF (bins of 1 ns) are neeed. We refer to these as TWF1ns. The TWF1ns are the input to the Simulation of the Energy Plane Response (SIERPE), which convolutes the TWF1ns with the response of the Energy plane FEE and outputs waveforms in bins of 25 ns (in adc counts), corresponding to those that are produced by the DAQ if no Baseline Restoration is performed in the FPGA. Those waveforms are called Raw Waveforms (RWF). RWF show the negative swing introduced by the FEE and need to be corrected by digital base line restoration (DBLR). 

3. DBLR can be implemented offline or online, in the DAQ FPGAs. DBLR takes RWF as input and produced Baseline restored waveforms (BRWF), where the negative swing has been corrected. 

http://localhost:8888/notebooks/Notebooks/SIERPE.ipynb

#### Raw Data

1. TWF, RWF and BRWF are different types of Raw Data (RD). 

2. For detailed studies of the EP FEE, TWF1ns are needed. The resulting RD files are large and need to be compressed.

3. For detailed studies of the DBLR, RWF are needed. The resulting RD files are smaller (bins of 25 ns), but still require compression.

4. Once DBLR is applied, either offline or in the FPGA, one produces BRWF. The RD files should include both RWF and BRWF for the EP, at least until DBLR in the FPGA is sufficiently tested. 

#### Raw Data flow in the MC

1. For detailed MC simulation of the EP one needs to produce Monte Carlo True Waveforms in bins of 1ns (TWF1ns).
2. TW1ns are passed by the SIERPE simulation which produces Raw Waveforms (RWF), showing negative swing.
3. RWF are passed by DBLR which produces Baseline restored Waveforms (BRWF)
4. Alternatively, NEXUS can produce True Waveforms in bins of 25 ns (TWF25ns).
5. TWF25ns are passed by a simplified simulation of the Energy plane response to produce directly BRWF.

#### Raw Data flow in DAQ

1. DAQ produced either RWF (if no DBLR in FPGA) or BRWF

#### DST files needed

1. MCTWF1ns : data files containing TWF1ns.
2. MCTWF25ns: data files containing TWF25ns.
3. RWF: data files containing MC or DAQ WF in bins of 25 ns before DBLR 
4. BRWF: data files containing MC or DAQ WF in bins of 25 ns after DBLR.

#### MC: From MCTWF1ns to BRWF

1. NEXUS + hdf5 converter to produce MCTWF1ns h5 files. 
2. DIOMIRA takes MCTWF1ns h5 files and produces RWF h5 files.
3. ISIDORA takes RWF and produces BRWF files which are added to the RWF h5 files.

#### DAQ: From RWF to BRWF

1. ISIDORA should also work on DAQ files

### Reconstructed Objects

1. Reconstructed objects (RO) are produced by sequencial processing of the data. 
2. In the EP, the relevant actions, starting from the BRWF are the calculation of the energy in each PMT (in pes) and the construction of time-series (after zero suppression) describing S1 and S2 signals
3. In the TP, the relevant actions is the construction of SiPM hits asociated to the S2 signals. 
4. RO need to be expressed in the Nh5 DST files. 